In [3]:
import zipfile
import sys
import os

from rdflib.term import Node

# sys.path.append(os.path.join(os.path.dirname(__file__), '..', '..'))
sys.path.append(os.path.join(os.path.abspath(os.path.join('..', '..'))))
from pipeline_generator.optimized_pipeline_generator import *


def connect_algorithms(ontology, algos_list):
    impls_algos = {imp : algo + "-Train" if "learner" in imp.fragment else algo
                   for algo in algos_list for (imp, _) in get_all_implementations(ontology, None, algo)}

    linked_impls = {}

    impls_list = list(impls_algos.keys())
    
    for i in range(len(impls_list) - 1):
        preceding_impl = impls_list[i]
        following_impls = impls_list[i + 1:]

        out_specs = get_implementation_output_specs(ontology, preceding_impl)
        out_spec_set = {out_sp for out_spec in out_specs for out_sp in out_spec}

        
        for following_impl in following_impls:

            in_specs = get_implementation_input_specs(ontology, following_impl)
            in_spec_set = {in_sp for in_spec in in_specs for in_sp in in_spec}

            if out_spec_set & in_spec_set:
                
                preceding_impl_key = impls_algos[preceding_impl]
                following_impl_key = impls_algos[following_impl]

                linked_impls.setdefault(preceding_impl_key, [])
                if following_impl_key not in linked_impls[preceding_impl_key]:
                    linked_impls[preceding_impl_key].append(following_impl_key)

    linked_impls[following_impl_key] = []


    return linked_impls


def abstract_planner(ontology: Graph, intent: Graph, visualization_parameters: Dict[str, List[str]]) -> Tuple[
    Dict[Node, Dict[Node, List[Node]]], Dict[Node, List[Node]], Dict[str, Union[str, List[str]]]]:
    dataset, task, algorithm, intent_iri = get_intent_info(intent)

    algs = [algorithm] if algorithm is not None else get_algorithms_from_task(ontology, task)
    # if algorithm is None:
    #     algs = get_algorithms_from_task(ontology, task)
    # else:
    #     algs = [algorithm]

    # print(f'ALGS: {algs}')
    impls = get_potential_implementations(ontology, task, algorithm)

    algs_shapes = {}
    alg_plans = {alg: [] for alg in algs}
    for impl in impls:
        alg = next(ontology.objects(impl[0], tb.implements)), 
        (impl[0], RDF.type, tb.Implementation) in ontology and (tb.ApplierImplementation not in ontology.objects(impl[0], RDF.type))

        algs_shapes[alg[0]] = impl[1::][0][0]

        alg_plans[alg[0]].append(impl)

    
    plans = {}
    for alg in algs:
        if cb.TrainTabularDatasetShape in algs_shapes[alg]:
            plans[alg] = connect_algorithms(ontology, [cb.DataLoading, cb.Partitioning, alg, cb.DataStoring])
            # trainer = cb.term(alg.fragment + '-Train')
            # plans[alg] = {
            #     cb.DataLoading: [cb.Partition],
            #     cb.Partition: [trainer, alg],
            #     trainer: [alg],
            #     alg: [cb.DataStoring],
            #     cb.DataStoring: []
            # }
        else:
            plans[alg] = connect_algorithms(ontology, [cb.DataLoading, alg])
            # plans[alg] = {
            #     cb.DataLoading: [alg],
            #     alg: [],
            # }

    print(f'ABS PLANS: {plans}')
    print(f'ALG PLANS: {alg_plans}')

    return plans, alg_plans, viz_params


ontology = get_ontology_graph()
intent_graph = get_graph_xp()
viz_params = {}

intent_name = 'ci'

intent_graph.add((ab.term(intent_name), RDF.type, tb.Intent))
intent_graph.add((ab.term(intent_name), tb.overData, ab.term('titanic.csv')))
# intent_graph.add((cb.Classification, tb.tackles, ab.term(intent_name)))
intent_graph.add((cb.DataVisualization, tb.tackles, ab.term(intent_name)))
intent_graph.add((cb.LinePlot, tb.solves, cb.DataVisualization))

intent = intent_graph

abstract_planner(ontology, intent, viz_params)
# print('---------------------------------------------------')

# print(f'ABS PLANS: {abs_plans}')
# print(f'ALG PLANS:{alg_plans}')

FOCUSED IMPL: [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#implementation-line_plot_visualizer')]
ABS PLANS: {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot'): {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataLoading'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot')], rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot'): []}}
ALG PLANS: {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot'): [(rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#implementation-line_plot_visualizer'), [[rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#NormalizedTabularDatasetShape'), rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#TabularDataset')]])]}


({rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot'): {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataLoading'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot')],
   rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot'): []}},
 {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#LinePlot'): [(rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#implementation-line_plot_visualizer'),
    [[rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#NormalizedTabularDatasetShape'),
      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#TabularDataset')]])]},
 {})

In [ ]:
# ABS PLANS: 
# {
#     rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree'): 
#     {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataLoading'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partitioning')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partitioning'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree-Train'), rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree-Train'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring'): []}, 
     
#     rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM'): 
#     {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataLoading'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partitioning')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partitioning'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM-Train'), rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM-Train'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring'): []}
# }



In [ ]:
# ABS PLANS:
# {
#     rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree'):
#     {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataLoading'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partition')], 
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partition'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree-Train'), rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree-Train'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DecisionTree'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring'): []},
     
#     rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM'):
#     {rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataLoading'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partition')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#Partition'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM-Train'), rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM-Train'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#SVM'): [rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring')],
#      rdflib.term.URIRef('https://extremexp.eu/ontology/cbox#DataStoring'): []}
# }
